# Statistical Methods in Image Processing EE-048954
## Homework 1: Kernel Density Estimation and Normalizing Flows
### Due Date: <span style="color:red">May 08, 2022</span>

###  Submission Guidelines

* Submission only in **pairs** on the course website (Moodle).
* Working environment:
    * We encourage you to work in `Jupyter Notebook` online using <a href="https://colab.research.google.com/">Google Colab</a> as it does not require any installation.
* You should submit two **separated** files:
    * A `.ipynb` file, with the name: `ee048954_hw1_id1_id2.ipynb` which contains your code implementations.
    * A `.pdf` file, with the name: `ee048954_hw1_id1_id2.pdf` which is your report containing plots, answers, and discussions.
    * **No handwritten submissions** and no other file-types (`.docx`, `.html`, ...) will be accepted.

### Mounting your drive for saving/loading stuff

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Importing relevant libraries for Part I

In [ ]:
## Standard libraries
import os
import math
import time
import numpy as np
import copy
import numpy as np
import numpy.linalg
from scipy.stats import multivariate_normal as mv_normal
from scipy.stats import norm
import pandas as pd
## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
plt.style.use('ggplot')
np.random.seed(0)
from scipy.signal import  convolve2d
import seaborn as sns
from scipy.stats import multivariate_normal
import xarray as xr
from pytictoc import TicToc
import pickle

## Part I: Kernel Density Estimation (30 points)

The multivariate kernel density estimate of a density $f(\mathbf{x})$ given a set of samples $\{\mathbf{x}_i\}$ is given by
$$\hat{f}(\mathbf{x}) = \frac{1}{N} \frac{1}{|H|} \sum_{i=1}^{N}  K(H^{-1} (\mathbf{x}_i-\mathbf{x})),
$$
where $H$ is a bandwidth matrix, $K(\cdot)$ is the kernel and $\{\mathbf{x}_i\}_{i=1}^N$ are $\textit{i.i.d.}$ samples drawn from $f(\mathbf{x})$.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 1</span>**. Consider the following density functions:
  * Gaussian Mixture:
  $$f(\mathbf{x};\sigma,\{\mu_i\}) = \frac{1}{2\pi\sigma^2} \sum_{m=1}^{M} \frac{1}{M} \exp\left\{-\frac{1}{2\sigma^2}||\mathbf{x}-\mathbf{\mu}_i||^2\right\} ,$$
   with $M = 4$, $\sigma = \frac{1}{2}$, and $\{\mathbf{\mu}_m\} = \{(0,0)^T , (0,2)^T , (2,0)^T , (2,2)^T\}$.
  * Gaussian Mixture with $M = 4$, $\sigma = 1$, and $\{\mathbf{\mu}_m\} = \{(0,0)^T , (0,2)^T , (2,0)^T , (2,2)^T\} $.
  * Spiral with $\theta \sim \mathcal{U}[0,4\pi]$ and $r|\theta \sim \mathcal{N}(\frac{\theta}{2},0.25)$.
  
  For each of the three distributions above, implement a function that draws $N = 10000$ samples ${\mathbf{x}_i}$ from $f(\mathbf{x})$. Display the drawn samples for each distribution separately.

In [ ]:
def set_visualization_settings():
    FIGURE_DPI = 300
    SAVEFIG_DPI = 96
    SMALL_FONT_SIZE = 14
    MEDIUM_FONT_SIZE = 16
    BIG_FONT_SIZE = 18
    TITLE_FONT_SIZE = 18
    SUPTITLE_FONT_SIZE = 22
    COLORS = ["darkblue", "darkgreen", "darkred"]
    plt.rcParams['figure.dpi'] = FIGURE_DPI
    plt.rcParams['savefig.dpi'] = SAVEFIG_DPI


    # Create an array with the colors to use
    # Set a custom color palette
    sns.set_palette(sns.color_palette(COLORS))

    plt.rc('font', size=SMALL_FONT_SIZE)  # controls default text sizes
    plt.rc('axes', titlesize=TITLE_FONT_SIZE)  # fontsize of the axes title
    plt.rc('axes', labelsize=BIG_FONT_SIZE)  # fontsize of the x and y labels
    plt.rc('axes', labelweight='bold')  # weight of the x and y labels
    plt.rc('xtick', labelsize=SMALL_FONT_SIZE)  # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_FONT_SIZE)  # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_FONT_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=SUPTITLE_FONT_SIZE)  # fontsize of the figure title
    # plt.rc('text', usetex=True)
    plt.rc('font', weight='bold')
    plt.rcParams['text.latex.preamble'] = r'\boldmath'
set_visualization_settings()
def clear_fname(fname):
    return fname.replace('{','').replace('}','').replace('$','').replace('\\','').replace('rm','').replace('^','').replace(',','').replace('_',' ')

def cart2pol(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    #theta = np.arctan(y, x)
    return(r, theta)

def pol2cart(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return(x, y)

def sample_from_gaussian_mixture(M,sigma,miu_m,N):
    """
    Sample Gaussian Mixture
    :param M:
    :param sigma:
    :param miu_m:
    :param N:
    :return:
    """
    gauss_components = np.random.choice(M, N ,replace=True)# sample N gaussian components
    xs= np.array([np.random.multivariate_normal(miu,sigma) for miu in miu_m[gauss_components]])
    pdf = 0
    for miu in miu_m:
        pdf+=mv_normal.pdf(xs,miu,sigma)
    samples = pd.DataFrame(xs,columns=['x','y'])
    return samples,pdf/M

def sample_from_spiral_dist(sigma,N):
    p_theta = 1/(4*np.pi)
    tetas = np.random.uniform(low=0,high=4*np.pi,size=N)
    rs = np.array([np.random.normal(teta/2,sigma) for teta in tetas])
    pdf=[]
    for teta,r in zip(tetas,rs):
        p_r_theta = norm.pdf(r,teta/2,sigma)
        p = p_theta *p_r_theta
        pdf.append(p)

    xs = [pol2cart(r,theta) for r,theta in zip(rs,tetas)]
    samples = pd.DataFrame(xs,columns=['x','y'])
    samples_rt = pd.DataFrame([[r,theta] for r,theta in zip(rs,tetas)],columns=['r','theta'])
    return samples,pdf, samples_rt

def draw_samples(samples,pdf_samples,figsize =(12, 9), s_title='samples',
                  elev = 45, azim = 12, label_x ='x',label_y='y',label_z='z',
                 save=False, fig_type='svg'):
    sns.set(style = "darkgrid")
    fig = plt.figure(figsize =figsize)
    ax = plt.axes(projection ='3d')
    values = ax.scatter3D(samples.x,samples.y,pdf_samples,c=pdf_samples,cmap='turbo',s=1)
    clb = fig.colorbar(values, ax = ax,shrink = 0.5, aspect = 5)#, orientation="horizontal")
    clb.ax.set_title(label_z)
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel(label_x)
    ax.set_ylabel(label_y)
    ax.zaxis.set_rotate_label(False)
    ax.set_zlabel(label_z,rotation=90)
    plt.tight_layout()
    ax.set_title(s_title)
    if save:
        f_name= clear_fname(s_title)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)

    # show plot
    plt.show()
    return fig,ax

def kernel(u):
    return np.exp(-u**2/2)/(np.sqrt(2*np.pi))

def plot_density(X,Y,Z,figsize =(15, 9), s_title='Z',
                elev = 45, azim = 12, label_x ='x',label_y='y',label_z='z',
                proj_offset=-0.05, samples=None, save=False, fig_type='svg', alpha=1.0):
    # Creating figure
    sns.set(style = "darkgrid")
    fig = plt.figure(figsize =figsize)
    ax = plt.axes(projection ='3d')

    # Creating plot
    surf = ax.plot_surface(X,Y,Z,
                        cmap = 'turbo',
                        edgecolor ='none',
                        alpha = alpha)
    clb = fig.colorbar(surf, ax = ax,shrink = 0.5, aspect = 5)
    clb.ax.set_title(label_z)
    ax.set_xlabel(label_x)
    ax.set_ylabel(label_y)
    ax.zaxis.set_rotate_label(False)
    ax.set_zlabel(label_z,rotation=90)
    ax.set_title(s_title)
    if samples is not None:
        ax.contourf(X, Y, Z, zdir='z', offset=proj_offset, cmap='turbo')
        # Adjust the limits, ticks and view angle
        ax.set_zlim(proj_offset,Z.max())
        ztick = np.linspace(start=0, stop=density.max()*1.01,num=6).tolist()
        ax.set_zticks(ztick,[f'{value:.2f}' for value in ztick])
        if samples is not None:
            ax.scatter(samples.x,samples.y,proj_offset,s=.05,c='white')
    ax.view_init(elev=elev, azim=azim)
    plt.tight_layout()

    if save:
        f_name= clear_fname(s_title)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)

    # show plot
    plt.show()
    return fig,ax

def calculate_GMV_pdf(X,Y,mus, sigmas):
    grid = np.dstack((X, Y))
    density = np.zeros(grid.shape[0:2])
    weight = 1/len(mus)
    for mu,signa in zip(mus,sigmas):
        rv = multivariate_normal(mu, sigma)
        z_i = np.reshape(rv.pdf(grid), grid.shape[0:2])
        density += z_i*weight
    return density


In [ ]:
# define grid
xlim = [-10.0,10.0]
ylim = [-10.0,10.0]
nx_grid = 400j
ny_grid = 400j
X, Y = np.mgrid[xlim[0]:xlim[1]:nx_grid, ylim[0]:ylim[1]:ny_grid]


fig_type='svg' # or 'png'
save_fig=False

In [ ]:
# Drawing samples of distributions
N = 10000
samples_1,pdf_1 = sample_from_gaussian_mixture(M=4,sigma= np.eye(2)*0.5,miu_m=np.array([[0,0],[0,2],[2,0],[2,2]]) ,N =N)
samples_2,pdf_2= sample_from_gaussian_mixture(M=4,sigma= np.eye(2)*1. ,miu_m=np.array([[0,0],[0,2],[2,0],[2,2]]) ,N =N)
samples_3,pdf_3,samples_rt = sample_from_spiral_dist(sigma=0.25,N=N)

for ind, (samples,pdf) in enumerate(zip([samples_1,samples_2,samples_3], [pdf_1,pdf_2,pdf_3])):
    name =rf'$f_{ind+1}$'
    draw_samples(samples,pdf,figsize =(10, 8), s_title='samples of '+name,label_z=r'$f(x,y)$',
                 elev = 35, azim = -40,save=save_fig,fig_type=fig_type)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 2</span>**. Implement a function that accepts samples $\{\mathbf{x}_i\}$ and a bandwidth matrix $ {H}$ and estimates $\hat{f}(\mathbf{x})$ using multivariate kernel density estimation. Use the two-dimension separable kernel $K( {u}) = k(u_1)k(u_2)$ where $k(u)= \frac{1}{\sqrt{2\pi}} \exp \{\frac{-u^2}{2}\}$.

In [ ]:
def KDE(X:np.ndarray,Y:np.ndarray,samples:pd.DataFrame,h:float):
    """
    KDE estimation of density, assuming gaussian kernel
    :param X: meshgrid of X
    :param Y: meshgrid of Y
    :param samples: pd.DataFrame with columns 'x' and 'y' for samples locations
    :param h: Kernel bandwidth (here we assume similar bandwidth for X and Y)
    :return: f_hat - The estimated density based on given samples
    """
    d = float(samples.shape[1]) # d - dimensions of samples
    N = float(samples.shape[0]) # N - number of samples

    # calculate kernel weights using 2D histogram
    d_x = X[1,0] - X[0, 0]
    d_y = Y[0,1] - Y[0, 0]
    edges_x = X[:,0]+.5*d_x
    edges_y = Y[0,:]+.5*d_y
    edge0_x = np.array(X[0, 0]-.5*d_x).reshape(1)
    edge0_y = np.array(Y[0, 0]-.5*d_y).reshape(1)
    edges_x = np.append(edge0_x,edges_x,axis=0)
    edges_y = np.append(edge0_y,edges_y,axis=0)
    kernel_weights, _, _ = np.histogram2d(x=samples.x, y=samples.y,
                                            bins=(edges_x, edges_y),density=False)
    # creating data array of weights, for debugging.
    kernel_weights_da = xr.DataArray(data=kernel_weights,coords={'x':X[:,0],'y':Y[0,:]},attrs={'long_name':r'$N_s$'})

    # setting separable kernel
    kernel_x =kernel(X/h)
    kernel_y =kernel(Y/h)
    K = kernel_x*kernel_y
    norm_factor = (h ** d) * N

    # Calculate KDE using 2D convolution
    f_hat = (1/norm_factor)*convolve2d(K, kernel_weights, mode="same")
    return f_hat, kernel_weights_da

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 3</span>**. For **each** distribution, compare between $f(\mathbf{x})$ and $\hat{f}(\mathbf{x})$ using the bandwidth matrices
${H} = \begin{pmatrix}
	h & 0\\0 & h
	\end{pmatrix}$ with $h = 0.1,0.5,1$. and display the estimation.
	Discuss the trade-off of the choice of $h$.

In [ ]:
LOAD_RESULTS = False  # change to True, if you wish to skip the calculations
KDE_fname= 'results_KDE.pickle'
if LOAD_RESULTS:
    print('Loading pre-calculated KDE results')
    with open(KDE_fname, 'rb') as handle:
        results = pickle.load(handle)
else:
    print('KDE estimation')
    # Estimate KDE for different h sizes
    BW = [.1,.5,1.0]
    results={}
    t = TicToc()
    for indf,(samples) in enumerate([samples_1,samples_2,samples_3]):
        func_name =rf'$f_{indf+1}$'
        results_ind={}
        for indh,(h) in enumerate(BW):
            t.tic()
            f_hat,kernel_weights_da = KDE(X, Y, samples,h)
            t.toc()
            print(f'for estimating f_{indf+1} using h={h}')
            results_ind[rf'h{indh}']=f_hat
        results_ind[rf'hist']=kernel_weights_da
        results[rf'f{indf}'] = results_ind

    # Save results of KDE
    with open(KDE_fname, 'wb') as handle:
        print('Saving KDE results')
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plot the estimated KDE for different h sizes
for indf,(samples) in enumerate([samples_1,samples_2,samples_3]):
    # plot f_hat for each h
    results_ind = results[rf'f{indf}']
    for indh,(h) in enumerate(BW):
        f_hat = results_ind[rf'h{indh}']
        plot_density(X,Y,f_hat,figsize =(10, 8),
                     s_title=fr"$\hat{{f}}_{indf+1}^{{\rm KDE}}\, for\, h={h},\, N={N}$",
                     label_z=r'$\hat{f}(x,y)$',alpha=1.0,
              elev = 35, azim = -40, save=save_fig, fig_type=fig_type)

    # plot kernel weights (Note that these weights are similar for any h. They are related to the grid of X,Y)
    kernel_weights_da = results_ind[rf'hist']
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize =(10,8))
    kernel_weights_da.plot(cmap='turbo',ax=ax)
    func_name =rf'$f_{indf+1}$'
    title_hist = fr'2D histogram of {func_name} samples'
    if save_fig:
        f_name= clear_fname(title_hist)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)
    ax.set_title(title_hist)
    plt.show()

In [ ]:
# Analyse KDE results of f1
indf=0
# calculating f1 on a dense grid
sigma= np.eye(2)*0.5
sigmas= [sigma,sigma,sigma,sigma]
mus=[[0,0],[0,2],[2,0],[2,2]]
density = calculate_GMV_pdf(X,Y,mus, sigmas)
title = rf'$f_{indf+1}$'
fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
                      samples=None,alpha=1.0,
                      save=save_fig, fig_type=fig_type)

# Comparing f1 with the resulted KDE's
results_ind = results[rf'f{indf}']
for indh,(h) in enumerate(BW):
    f_hat = results_ind[rf'h{indh}']
    f_diff = np.sqrt((density-f_hat)**2)
    title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
    fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
                      samples=None, alpha=1.0,
                      save=save_fig, fig_type=fig_type)

In [ ]:
# Analyse KDE results of f2
indf=1
# calculating f2 on a dense grid
sigma= np.eye(2)*1
sigmas= [sigma,sigma,sigma,sigma]
mus=[[0,0],[0,2],[2,0],[2,2]]
density = calculate_GMV_pdf(X,Y,mus, sigmas)
title = rf'$f_{indf+1}$'
fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
                      samples=None,alpha=1.0,
                      save=save_fig, fig_type=fig_type)

# Comparing f1 with the resulted KDE's
results_ind = results[rf'f{indf}']
for indh,(h) in enumerate(BW):
    f_hat = results_ind[rf'h{indh}']
    f_diff = np.sqrt((density-f_hat)**2)
    title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
    fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
                      samples=None, alpha=1.0,
                      save=save_fig, fig_type=fig_type)


In [ ]:
# Analyse KDE results of f3 ?
# indf=2
# # calculating f3 on a dense grid......Not working :-/
# R,THETA = cart2pol(X,Y)
# THETA += np.pi
# SIGMAs = np.ones_like(THETA)*0.25
# THETA_ = 2*R
# MUs = THETA/2
# f_r_theta = np.random.normal(MUs,SIGMAs)
# plt.imshow(f_r_theta)
# #density = calculate_GMV_pdf(X,Y,mus, sigmas)
# title = rf'$f_{indf+1}$'
# fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
#                   elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
#                       samples=None,alpha=1.0,
#                       save=save_fig, fig_type=fig_type)
#
# # Comparing f1 with the resulted KDE's
# results_ind = results[rf'f{indf}']
# for indh,(h) in enumerate(BW):
#     f_hat = results_ind[rf'h{indh}']
#     f_diff = np.sqrt((density-f_hat)**2)
#     title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
#     fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
#                   elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
#                       samples=None, alpha=1.0,
#                       save=save_fig, fig_type=fig_type)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 4</span>**. Which of the distributions was the easiest/hardest to estimate? Why?